In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import Row
from pyspark import StorageLevel
from pyspark.sql import SparkSession
import sys
import numpy as np

In [2]:
# sc = SparkContext("local[*]", "NFL")
# rdd = sc.textFile('../in/PlayerTrackData.csv')

# remove header
# rdd = rdd.filter(lambda x: 'PlayKey' not in x)

# split the col
# rdd = rdd.map(lambda x: x.split(','))

# remove row if missing some value
# rdd = rdd.filter(lambda x: x[5] != '' and x[7] != '')

# convert to dataframe object
# sqlContext = SQLContext(sc)
# df = rdd.toDF(['PlayKey','time','event','x','y','dir','dis','o','s'])

# if df want to use rdd function
# just df.rdd.xxx()

In [3]:
spark = SparkSession.builder.master("local[*]").appName("NFL").getOrCreate()
tracks = spark.read.option("header",True).csv('../in/PlayerTrackData.csv')

In [6]:
# show header
# rdd.take(1)
# show schema
tracks.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: string (nullable = true)
 |-- s: string (nullable = true)



In [8]:
# convert the defautl datatype
tracks = tracks.withColumn("time", tracks.time.cast('float').alias("time"))
tracks = tracks.withColumn("x", tracks.x.cast('float').alias("x"))
tracks = tracks.withColumn("y", tracks.y.cast('float').alias("y"))
tracks = tracks.withColumn("dir", tracks.dir.cast('float').alias("dir"))
tracks = tracks.withColumn("o", tracks.o.cast('float').alias("o"))
tracks = tracks.withColumn("s", tracks.s.cast('float').alias("s"))
tracks = tracks.withColumn("x", tracks.x.cast('float').alias("x"))

In [9]:
tracks.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: float (nullable = true)
 |-- event: string (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- dir: float (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: float (nullable = true)
 |-- s: float (nullable = true)



In [10]:
# get all ball_snap play record
ball_snap_tracks = tracks.where("event == 'ball_snap'")

# rename x,y,time for ball snap
drop_list = ['dir','dis','o','event','s']
ball_snap_tracks = ball_snap_tracks.withColumnRenamed('x','snap_x')\
.withColumnRenamed('y','snap_y')\
.withColumnRenamed('time','snap_time').drop(*drop_list)

# join two table (orign tracks and tracks only contained ball_snap event)
tracks = tracks.join(ball_snap_tracks, ['PlayKey'], "left_outer")

# remove any data for a play 0.1 second before snap
tracks = tracks.where('time >= (snap_time - 0.1)')


In [11]:
# get the last event of each play key
last_event_tracks = tracks.where('event is not null').groupBy('PlayKey').agg(f.max('time').alias('time'))

# join last event tracks and tracks
# only get the playkey, time, and event field
last_event_tracks = last_event_tracks.join(tracks, ['PlayKey','time'], "inner")[['PlayKey','time','event']]

# rename the column
last_event_tracks = last_event_tracks.withColumnRenamed('time','last_time').withColumnRenamed('event','last_event')

# park the result of last event tracks to the tracks
tracks = tracks.join(last_event_tracks, ['PlayKey'], 'left_outer')

# Remove any data for a play 0.1 second after last event
tracks = tracks.where('time <= (last_time + 0.1)')

In [12]:
tracks.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: float (nullable = true)
 |-- event: string (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- dir: float (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: float (nullable = true)
 |-- s: float (nullable = true)
 |-- snap_time: float (nullable = true)
 |-- snap_x: float (nullable = true)
 |-- snap_y: float (nullable = true)
 |-- last_time: float (nullable = true)
 |-- last_event: string (nullable = true)



In [13]:
# save tracks into memory
# only record the time from the start of first event-1 to the end of last event+1
tracks.persist(StorageLevel.MEMORY_ONLY)

DataFrame[PlayKey: string, time: float, event: string, x: float, y: float, dir: float, dis: string, o: float, s: float, snap_time: float, snap_x: float, snap_y: float, last_time: float, last_event: string]

In [14]:
tracks.show()
print(tracks.count())

+-----------+----+---------+-----+-----+------+----+-----+----+---------+------+------+---------+--------------------+
|    PlayKey|time|    event|    x|    y|   dir| dis|    o|   s|snap_time|snap_x|snap_y|last_time|          last_event|
+-----------+----+---------+-----+-----+------+----+-----+----+---------+------+------+---------+--------------------+
|26624-13-32|19.5|ball_snap|19.85| 23.9|266.62| 0.0|15.32|0.51|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.6|     null|19.84| 23.9|266.04| 0.0|15.48|0.68|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.7|     null|19.84| 23.9|265.88| 0.0|15.85| 0.8|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.8|     null|19.84| 23.9|265.98| 0.0|16.76|0.89|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|19.9|     null|19.83| 23.9|266.15|0.01|18.18|0.96|     19.5| 19.85|  23.9|     23.6|pass_outcome_inco...|
|26624-13-32|20.0|     null|19.82| 23.9|266.43|0

In [28]:
#load play list data
playlist = spark.read.option("header",True).csv('../in/PlayList.csv')

# convert player day to integer
playlist = playlist.withColumn("PlayerDay", playlist.PlayerDay.cast('int').alias("PlayerDay"))

In [29]:
playlist.printSchema()

root
 |-- PlayerKey: string (nullable = true)
 |-- GameID: string (nullable = true)
 |-- PlayKey: string (nullable = true)
 |-- RosterPosition: string (nullable = true)
 |-- PlayerDay: integer (nullable = true)
 |-- PlayerGame: string (nullable = true)
 |-- StadiumType: string (nullable = true)
 |-- FieldType: string (nullable = true)
 |-- Temperature: string (nullable = true)
 |-- Weather: string (nullable = true)
 |-- PlayType: string (nullable = true)
 |-- PlayerGamePlay: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- PositionGroup: string (nullable = true)



In [41]:
# get season 2 distinct player key
tough_playerkey = playlist.select('PlayerKey').where('PlayerDay >= 350').distinct()
tough_playerkey = tough_playerkey.rdd.map(lambda x:x.PlayerKey).collect()

In [42]:
# only select the player who play in season 2, dont know is he play season 1
tough_playlist = playlist[playlist.PlayerKey.isin(season2_playerkey)]

# add season to df, if less than 350, season 1 else season 2
tough_playlist = season2_playlist.withColumn('season', f.when(f.col("PlayerDay") < 350,1).otherwise(2))

In [43]:
games = tough_playlist[['GameID','season']].distinct()

In [48]:
_tracks = tracks.join(tough_playlist[['PlayKey','GameID']],['PlayKey'],'left_outer')
_tracks = _tracks.join(games,['GameID'],'left_outer')
_tracks.persist(StorageLevel.MEMORY_ONLY)

DataFrame[GameID: string, PlayKey: string, time: float, event: string, x: float, y: float, dir: float, dis: string, o: float, s: float, snap_time: float, snap_x: float, snap_y: float, last_time: float, last_event: string, season: int]

In [52]:
# Fix orientation
# Assume other seasons based on direction at snap - if orientation at snap is outside normal range, shift.
s1 = _tracks.select('PlayKey').where('event == "ball_snap" and o < 50 and season is null').distinct().collect()
s2 = _tracks.select('PlayKey').where('event == "ball_snap" and o > 325 and season is null').distinct().collect()
s3 = _tracks.select('PlayKey').where('event == "ball_snap" and o < 225 and o > 125 and season is null').distinct().collect()

def get(x, field='PlayKey'):
    return [i[field] for i in x]

abnormal_playkey = get(s1)+get(s2)+get(s3)
abnormal_playkey = list(set(abnormal_playkey))

In [55]:
# all abnormal orientation record will be treated as session 1
tracks = _tracks.withColumn('season', 
                            f.when(_tracks.season.isNull()
                                   & _tracks.PlayKey.isin(abnormal_playkey),1).otherwise(f.col('season')))

# all normal orientation record will be treated as session 2
tracks = tracks.withColumn('season', 
                           f.when(tracks.season.isNull()
                                  & ~tracks.PlayKey.isin(abnormal_playkey),2).otherwise(f.col('season')))
tracks.persist(StorageLevel.MEMORY_ONLY)

In [60]:
# Change orientation for season 1
clean_tracks = tracks.withColumn('o',f.when(f.col('season')== 1, (f.col('o')+90%360)).otherwise(f.col('o')))
clean_tracks.persist(StorageLevel.MEMORY_ONLY)

DataFrame[GameID: string, PlayKey: string, time: float, event: string, x: float, y: float, dir: float, dis: string, o: float, s: float, snap_time: float, snap_x: float, snap_y: float, last_time: float, last_event: string, season: int]

In [70]:
# join clean track and playlist
tracks = clean_tracks.join(playlist[['PlayKey','PlayerKey','RosterPosition','PositionGroup','FieldType','PlayType']], ['PlayKey'], 'inner')

In [71]:
#load injury data
injury = spark.read.option("header",True).csv('../in/InjuryRecord.csv')
injury_plays = injury.select('PlayKey').distinct().collect()
injury_plays = get(injury_plays)

In [ ]:
# only focus on 3 kind of position
tracks = tracks.withColumn('isInjuryPronePos', 
                           f.when(tracks.PositionGroup.isin(['DB','WR','LB']),True).otherwise(False))

tracks = tracks.withColumn('isInjuryPlay', 
                           f.when(tracks.PositionGroup.isin(injury_plays),True).otherwise(False))

tracks = tracks.withColumn('isRushPass', 
                           f.when(tracks.PositionGroup.isin(injury_plays),True).otherwise(False))

tracks = tracks.withColumn('isInjuredPlayer', 
                           f.when(tracks.PositionGroup.isin(injury_plays),True).otherwise(False))



In [ ]:
# only focus on 3 kind of position
tracks = tracks.withColumn('isInjuryPronePos', 
                           f.when(tracks.PositionGroup.isin(['DB','WR','LB']),True).otherwise(False))

tracks = tracks.withColumn('isInjuryPlay', 
                           f.when(tracks.PositionGroup.isin(['DB','WR','LB']),True).otherwise(False))



In [100]:
# speed
# player_mean_speed_df = df.rdd.map(lambda x: [x[0].split('-')[0], float(x[-1])]).toDF(["playerKey","s"])
# # remove extreme small speed
# player_mean_speed_df = player_mean_speed_df.rdd.filter(lambda x:x.s >= 1).toDF(["playerKey","s"])
# # get average speed
# player_mean_speed = player_mean_speed_df.groupBy('playerKey').agg(f.mean("s").alias('mean'))
# result = player_mean_speed.collect()

In [88]:
# convert string to float
df = df.withColumn("s", df.s.cast('float').alias("s"))
df = df.withColumn("o", df.o.cast('float').alias("o"))
df = df.withColumn("dir", df.dir.cast('float').alias("dir"))

In [89]:
df.printSchema()

root
 |-- PlayKey: string (nullable = true)
 |-- time: string (nullable = true)
 |-- event: string (nullable = true)
 |-- x: string (nullable = true)
 |-- y: string (nullable = true)
 |-- dir: float (nullable = true)
 |-- dis: string (nullable = true)
 |-- o: float (nullable = true)
 |-- s: float (nullable = true)



In [90]:
# add new col.
# compute the difference between o and dir
def add_o_dir_diff(x):
    x = x.asDict()
    x['o_dir_diff'] = min([abs(x['o']-x['dir']),abs(x['o']-(x['dir']-360)),abs(x['o']-(x['dir']+360))])
    
    x['is_forward_move'] = False
    x['is_sideway_move'] = False
    x['is_backward_move'] = False
    
    if x['o_dir_diff'] <= 45:
        x['move_type'] = 'forward'
        x['is_forward_move'] = True
    elif x['o_dir_diff'] <= 135:
        x['move_type'] = 'sideway'
        x['is_sideway_move'] = True
    else:
        x['move_type'] = 'backward'
        x['is_backward_move'] = True
        
    return Row(**x)

rdd = df.rdd.map(add_o_dir_diff)

In [91]:
df = rdd.toDF()

In [99]:
# get the distinct event in player trace data
# res = df.select('event').distinct().collect()

In [101]:
# we only focus on ball-snap (the most common event in player trace data)
# rename some columns
_df = df.rdd.filter(lambda x: x.event == 'ball-snap').toDF()
_df = _df.withColumnRenamed('x','x_snap').withColumnRenamed('y','y_snap').withColumnRenamed('time','time_snap')

KeyboardInterrupt: 

In [ ]:
# Remove any data for a play 0.1 second before snap
df = df.merge(_df, on='PlayKey', how='left')
df = df.rdd.filter(lambda x: x.time >= x.time_snap-0.1)

# Remove any data for a play 0.1 second after snap
# TODO